In [1]:
import requests
import pandas as pd
import json

In [2]:
def load_api_key(key_path: str):
    with open(key_path, mode='r') as file:
        API_KEY = file.read()
    return API_KEY


def get_sports_data(api_key: str, return_data: bool = False, return_headers: bool = False):

    sports_response = requests.get('https://api.the-odds-api.com/v4/sports', params={
        'api_key': api_key
    })

    if sports_response.status_code != 200:
        print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')
    else:
        # write sports data to database in "in_season_sports" table
        df = pd.DataFrame(sports_response.json())
        # write headers info to database in "in_season_sports_headers" table
        headers = sports_response.headers
        headers_df = pd.DataFrame(data=dict(headers), index=[0])
    
    if return_data:
        return df
    if return_headers:
        return headers_df
    
    return df, headers_df


def get_odds_data(api_key: str, sport: str, regions: str, markets: str, odds_format: str, date_format: str):

    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{sport}/odds', params={
        'api_key': api_key,
        'regions': regions,
        'markets': markets,
        'oddsFormat': odds_format,
        'dateFormat': date_format,
    })

    with open(r'/home/carls/projects/odds_site/odds_site_db/epl_odds_response.json', mode='w') as json_file:
        odds_json = json.dump(odds_response.json(), json_file)

    extra_meta = [['bookmakers', 'key'], ['bookmakers', 'title'], ['bookmakers', 'last_update'], ['bookmakers', 'markets', 'last_update'], ['bookmakers', 'markets', 'key']]
    meta = pd.json_normalize(data=odds_response.json()).columns.to_list()[:-1] + extra_meta

    odds_df = pd.json_normalize(
        data=odds_response.json(),
        meta=meta,
        record_path=['bookmakers', 'markets', 'outcomes'],
    )

    odds_headers_df = pd.DataFrame(dict(odds_response.headers), index=[0])

    return odds_df, odds_headers_df, odds_response

In [3]:
DB_PATH = r'/home/carls/projects/odds_site/odds_site_db/'
BASE_URL = 'https://api.the-odds-api.com/v4/sports'

APIKEY_PATH = r'/home/carls/projects/odds_site/api_key/odds_api_key.txt'
API_KEY = load_api_key(key_path = APIKEY_PATH)

In [4]:
SPORT = 'soccer_epl'
REGIONS = 'us'
MARKETS = 'h2h'
ODDS_FORMAT = 'decimal'
DATE_FORMAT = 'iso'

data, headers, res = get_odds_data(
    api_key=API_KEY, 
    sport=SPORT,
    regions=REGIONS,
    markets=MARKETS,
    odds_format=ODDS_FORMAT,
    date_format=DATE_FORMAT
)

In [5]:
data

,name,price,id,sport_key,sport_title,commence_time,home_team,away_team,bookmakers.key,bookmakers.title,bookmakers.last_update,bookmakers.markets.last_update,bookmakers.markets.key
0,Aston Villa,2.75,d13dd3180de59bb9a0c3721d7a045d99,soccer_epl,EPL,2023-04-04T18:45:00Z,Leicester City,Aston Villa,fanduel,FanDuel,2023-04-04T14:50:36Z,2023-04-04T14:50:36Z,h2h
1,Leicester City,2.55,d13dd3180de59bb9a0c3721d7a045d99,soccer_epl,EPL,2023-04-04T18:45:00Z,Leicester City,Aston Villa,fanduel,FanDuel,2023-04-04T14:50:36Z,2023-04-04T14:50:36Z,h2h
2,Draw,3.30,d13dd3180de59bb9a0c3721d7a045d99,soccer_epl,EPL,2023-04-04T18:45:00Z,Leicester City,Aston Villa,fanduel,FanDuel,2023-04-04T14:50:36Z,2023-04-04T14:50:36Z,h2h
3,Aston Villa,2.70,d13dd3180de59bb9a0c3721d7a045d99,soccer_epl,EPL,2023-04-04T18:45:00Z,Leicester City,Aston Villa,williamhill_us,William Hill (US),2023-04-04T14:51:33Z,2023-04-04T14:51:33Z,h2h
4,Leicester City,2.55,d13dd3180de59bb9a0c3721d7a045d99,soccer_epl,EPL,2023-04-04T18:45:00Z,Leicester City,Aston Villa,williamhill_us,William Hill (US),2023-04-04T14:51:33Z,2023-04-04T14:51:33Z,h2h
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105,Liverpool,1.77,8082321e458b3bf57c62a9c2648ace85,soccer_epl,EPL,2023-04-17T19:00:00Z,Leeds United,Liverpool,pointsbetus,PointsBet (US),2023-04-04T14:52:38Z,2023-04-04T14:52:38Z,h2h
1106,Draw,4.10,8082321e458b3bf57c62a9c2648ace85,soccer_epl,EPL,2023-04-17T19:00:00Z,Leeds United,Liverpool,pointsbetus,PointsBet (US),2023-04-04T14:52:38Z,2023-04-04T14:52:38Z,h2h
1107,Leeds United,4.10,8082321e458b3bf57c62a9c2648ace85,soccer_epl,EPL,2023-04-17T19:00:00Z,Leeds United,Liverpool,foxbet,FOX Bet,2023-04-04T14:50:36Z,2023-04-04T14:50:36Z,h2h
1108,Liverpool,1.75,8082321e458b3bf57c62a9c2648ace85,soccer_epl,EPL,2023-04-17T19:00:00Z,Leeds United,Liverpool,foxbet,FOX Bet,2023-04-04T14:50:36Z,2023-04-04T14:50:36Z,h2h


In [6]:
headers

,Date,Content-Type,Content-Length,Connection,vary,content-encoding,X-Requests-Used,X-Requests-Remaining,Apigw-Requestid
0,"Tue, 04 Apr 2023 14:53:14 GMT",application/json; charset=utf-8,5180,keep-alive,Accept-Encoding,gzip,2,498,C236LilAoAMEP8w=


In [9]:
res.json()

[{'id': 'd13dd3180de59bb9a0c3721d7a045d99',
  'sport_key': 'soccer_epl',
  'sport_title': 'EPL',
  'commence_time': '2023-04-04T18:45:00Z',
  'home_team': 'Leicester City',
  'away_team': 'Aston Villa',
  'bookmakers': [{'key': 'fanduel',
    'title': 'FanDuel',
    'last_update': '2023-04-04T14:50:36Z',
    'markets': [{'key': 'h2h',
      'last_update': '2023-04-04T14:50:36Z',
      'outcomes': [{'name': 'Aston Villa', 'price': 2.75},
       {'name': 'Leicester City', 'price': 2.55},
       {'name': 'Draw', 'price': 3.3}]}]},
   {'key': 'williamhill_us',
    'title': 'William Hill (US)',
    'last_update': '2023-04-04T14:51:33Z',
    'markets': [{'key': 'h2h',
      'last_update': '2023-04-04T14:51:33Z',
      'outcomes': [{'name': 'Aston Villa', 'price': 2.7},
       {'name': 'Leicester City', 'price': 2.55},
       {'name': 'Draw', 'price': 3.35}]}]},
   {'key': 'mybookieag',
    'title': 'MyBookie.ag',
    'last_update': '2023-04-04T14:53:09Z',
    'markets': [{'key': 'h2h',
     